# 1.Configuracion del Ambiente


In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# 2.Importar Verbos

In [2]:
import pickle
import requests

# URLs de los archivos en GitHub
url_lista_verbos = "https://github.com/JorgeHdzRiv/Desafios_Alura_DS/raw/refs/heads/main/ChatBot/models/lista_verbos.pickle"
url_verbos_irregulares = "https://github.com/JorgeHdzRiv/Desafios_Alura_DS/raw/refs/heads/main/ChatBot/models/verbos_irregulares.pickle"

# Función para descargar y cargar archivos pickle desde una URL
def cargar_pickle_desde_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Verificar si la solicitud fue exitosa
    return pickle.loads(response.content)

# Cargar los archivos en variables
lista_verbos = cargar_pickle_desde_url(url_lista_verbos)
verbos_irregulares = cargar_pickle_desde_url(url_verbos_irregulares)

# Mostrar resultados
print(lista_verbos)
print(verbos_irregulares)


['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

# 3.Tratamiento de Datos

In [3]:
# Ejemplo de jellyfish
jellyfish.jaro_winkler_similarity('sentirme','sentir')

0.95

## 3.1 Raiz de las palabras

Usando este ejemplo, crearemos una función raiz que reciba una palabra y la compare con todas las palabras de la lista_verbos utilizando jaro_winkler, y que devuelva la palabra de lista_verbos con mayor similaridad a la palabra ingresada.

Observación: Si la palabra encontrada, con mayor similaridad, no supera el radio de 0.93 entonces deberá regresar la palabra original.

El objetivo de esta función es encontrar la raíz del verbo ingresado.

In [4]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  max_similaridad = 0
  mejor_coincidencia = palabra #Por defecto, devolvemos la palabra original

  for verbo in lista_verbos:
    similitud = jellyfish.jaro_winkler_similarity(palabra, verbo)
    if similitud > max_similaridad:
      max_similaridad = similitud
      mejor_coincidencia = verbo

  return mejor_coincidencia if max_similaridad >= 0.93 else palabra

# Ejemplo de uso
print(raiz("sentirme"))  # Debería devolver "sentir" si está en lista_verbos
print(raiz("comiendo"))  # Si "comer" está en lista_verbos, lo devuelve
print(raiz("dormir"))  # Debería devolver "dormir" si no está en lista_verbos

sentir
comiendo
dormir


## 3.2 Tratamiento de texto

Crea una función tratamiento_texto que reciba una frase de texto y devuelva la misma frase pero sin acentuaciones, todo en minúscula y sin espacios en blanco adicionales.

El objetivo de esta función es equilibrar los textos

In [5]:
def tratamiento_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

    # Eliminar espacios adicionales
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

# Ejemplo de uso
print(tratamiento_texto('¡Buen día! ¿Cómo está todo hoy?'))

¡buen dia! ¿como esta todo hoy?


## 3.3 Reemplazar terminacion

Crear una función **reemplazar_terminacion** que reciba una palabra e identifique si la misma termina en alguna de las siguientes palabras: “es”, “me”, “as”, “te”, “ste”, si coincide, entonces que substituya esa terminación por la letra “r”.

El objetivo de esta función es aproximar el verbo a su raíz.

In [6]:
# Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
    terminaciones = ["ste","es", "me", "as", "te"]

    for terminacion in terminaciones:
        if palabra.endswith(terminacion):
            return palabra[:-len(terminacion)] + "r"

    return palabra  # Si no coincide, devuelve la palabra original

# Ejemplo de uso
print(reemplazar_terminacion("comes"))    # "comer"
print(reemplazar_terminacion("sentirme")) # "sentir"
print(reemplazar_terminacion("hablaste")) # "hablar"
print(reemplazar_terminacion("viniste"))  # "venir"

comr
sentirr
hablar
vinir


## 3.4 Normalizar Texto

Esta función se aprovecha de todas las funciones que has creado anteriormente para regresar una frase a su raíz original, a esto se le conoce como normalizar textos, que es fundamental para el aprendizaje de máquinas, además de aprovechar tus funciones, también utiliza la biblioteca spacy para identificar el tipo de palabra que compone una frase y así decidir cuales debe mantener y cuales palabras deberá eliminar por no ser importantes.

Ejemplo de uso de atributos doc

In [7]:
frase = 'Yo soy Jorge y me gusta el deporte como el futbol'
doc = nlp(frase)
for t in doc:
  print(t.text, '-', t.pos_, '-', t.lemma_)

Yo - PRON - yo
soy - AUX - ser
Jorge - PROPN - Jorge
y - CCONJ - y
me - PRON - yo
gusta - VERB - gustar
el - DET - el
deporte - NOUN - deporte
como - SCONJ - como
el - DET - el
futbol - NOUN - futbol


In [8]:
#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

# 4.Cargar base de documentos

## 4.1 Descargar base de dialogos

In [9]:
# URL directa del archivo ZIP en GitHub
zip_url = "https://github.com/JorgeHdzRiv/Desafios_Alura_DS/raw/refs/heads/main/ChatBot/data/dialogos.zip"

# Descargar el archivo ZIP
os.system(f"wget {zip_url} -O dialogos.zip")

# Descomprimir la carpeta
os.system("unzip -o dialogos.zip")

# Eliminar el archivo ZIP después de extraer
os.system("rm dialogos.zip")

# Verificar archivos descargados
os.system("ls ./dialogos")

0

In [10]:
# Importando bases de diálogo fluido
txt_folder_path = './dialogos'
lista_documentos = [x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]

# Listas para almacenar preguntas, respuestas y tipos de diálogos
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [], [], []

# Procesar cada archivo de diálogo
for idx in range(len(lista_documentos)):
    tipo = lista_documentos[idx].replace('.txt', '')  # Extraer tipo del archivo

    with open(os.path.join(txt_folder_path, lista_documentos[idx]), 'r', encoding='utf-8', errors='ignore') as f:
        lineas = [line.strip() for line in f.readlines() if line.strip()]  # Eliminar líneas vacías

        # Procesar el archivo línea por línea (pregunta-respuesta en pares)
        for i in range(0, len(lineas) - 1, 2):  # Itera de 2 en 2 (pregunta -> respuesta)
            pregunta = re.sub(r"[^\w\s+\-*/]", '', lineas[i])  # Limpiar pregunta
            pregunta = tratamiento_texto(pregunta)  # Aplicar tratamiento de texto
            respuesta = lineas[i + 1]  # La respuesta no necesita tratamiento especial

            # Agregar a las listas
            lista_dialogos.append(pregunta)
            lista_dialogos_respuesta.append(respuesta)
            lista_tipo_dialogo.append(tipo)

# Creando DataFrame de diálogos
datos = {
    'dialogo': lista_dialogos,
    'respuesta': lista_dialogos_respuesta,
    'tipo': lista_tipo_dialogo,
    'interseccion': 0,
    'jaro_winkler': 0,
    'probabilidad': 0
}

df_dialogo = pd.DataFrame(datos)

# Eliminar duplicados y resetear índice
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

In [11]:
df_dialogo.shape

(1124, 6)

In [12]:
df_dialogo.sample(5)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
361,me alegra haberte podido ayudar hasta luego,¡Gracias por todo! ¡Hasta luego!,Despedida,0,0,0
767,hola como estas hoy,"Estoy bien, gracias por preguntar. ¿En qué pue...",Saludos,0,0,0
543,puedes explicar el concepto de odio,El odio es un sentimiento de aversión o repuls...,Sentimiento,0,0,0
460,me dijiste que eres una persona humana,"Bueno, ¡si tú lo dices!",Identidad,0,0,0
320,me puedes ayudar con una tarea,"No, lo sentimos, no podemos ayudarte con tus t...",Otros,0,0,0


# 5.Buscar respuestas del chatbot

## 5.1 Comparacion de textos

Tenemos nuestra funcion **dialogo()** ,esta función recibe la pregunta del usuario, le realiza una limpieza, y luego recorre todo el dataframe df_dialogo para comparar la pregunta del usuario con la pregunta de los diálogos, si encuentra alguna pregunta que se parezca en más de un 93% entonces devuelve la respuesta correspondiente sino devuelve en blanco.

La misión en esta sección será crear 3 tipos de comparación de texto, entre la pregunta del usuario y la pregunta del diálogo (columna dialogo), y devolver el porcentaje de similaridad (entre 0 y 1) de esta comparación, este número será guardado en las columnas 'interseccion', 'similarity', 'jaro_winkler' de nuestro dataframe, correspondiendo al método usado en la comparación:

In [13]:
# Método 1: Intersección de palabras
def interseccion(text1, text2):
    palabras_text1 = set(text1.split())
    palabras_text2 = set(text2.split())
    return len(palabras_text1 & palabras_text2) / max(1, len(palabras_text1))  # Evitar división entre 0

interseccion('hola como vas','hola como estas mi amigo')

0.6666666666666666

In [14]:
# Método 2: Similaridad con TfidfVectorizer + Cosine Similarity
vectorizer = TfidfVectorizer()

def similarity(text1, text2):
    tfidf_matrix = vectorizer.fit_transform([text1, text2])  # Solo usa text1 y text2
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

# Prueba del método
similarity('hola como vas', 'hola como estas mi amigo')

0.35630042933313816

In [15]:
# Método 3: Jaro-Winkler Similarity
def jaro_winkler(text1, text2):
    return jellyfish.jaro_winkler_similarity(text1, text2)

jaro_winkler('hola como vas', 'hola como estas mi amigo')

0.867948717948718

In [31]:
# Función para verificar si el usuario inició un diálogo
def dialogo(user_response):
    # Tratamiento del texto del usuario
    user_response = tratamiento_texto(user_response)  # Normalización del texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()  # Copia del DataFrame

    # Asegurar que todas las columnas existen en el DataFrame
    columnas_faltantes = {'interseccion', 'jaro_winkler', 'probabilidad', 'similarity'} - set(df.columns)
    for col in columnas_faltantes:
        df[col] = 0.0  # Inicializar como float

    # Convertir columnas a float64 para evitar advertencias
    df[['interseccion', 'jaro_winkler', 'probabilidad', 'similarity']] = df[
        ['interseccion', 'jaro_winkler', 'probabilidad', 'similarity']
    ].astype(float)

    for idx, row in df.iterrows():
        # Comparaciones de texto
        df.at[idx, 'interseccion'] = float(interseccion(user_response, row['dialogo']))
        df.at[idx, 'similarity'] = float(similarity(user_response, row['dialogo']))  # Se añadió 'similarity'
        df.at[idx, 'jaro_winkler'] = float(jaro_winkler(user_response, row['dialogo']))

        # Determinar la mayor probabilidad entre los métodos
        df.at[idx, 'probabilidad'] = float(max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler']))

    # Ordenar por la mejor coincidencia
    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)
    probabilidad = df['probabilidad'].head(1).values[0]

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''

    return respuesta


In [32]:
# Poniendo a prueba la funcion dialogo
respuesta = dialogo('dame un codigo')
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0000000000000002
No estoy programado para ayudarte con esto, sólo estoy autorizado a responder tus dudas sobre Ciencia de Datos


## 5.2 Machine learning (DESARROLLO)



Utilizar comparación de textos para encontrar la respuesta es bastante útil, pero utilizar un modelo entrenado de Machine Learning es mucho mejor, veamos como podemos entrenar el nuestro:

Entre los mejores modelos para clasificación de texto, esto es, recibe una frase de entrada y devuelve el tipo de esta frase, tenemos los siguientes:

**Naive Bayes (Básico)**

**Random Forest (Intermedio)**

**Transformers (Avanzado)**

Usaremos el modelo Transformers que previamente fue entrenado de acuerdo a nuestro DataFrame

Por último, en la función clasificacion_modelo()deberás usar tu modelo entrenado para predecir la frase ingresada por el usuario, una vez identificado el tipo de la frase, el algoritmo buscará la pregunta más parecida de este tipo en nuestro df_dialogo y devolverá su respectiva respuesta, en caso el tipo sea diferente de ‘Otros’ y su similaridad mayor a 0.5, sino devolverá en blanco.

In [21]:
# URL directa del archivo ZIP en GitHub
zip_url = "https://github.com/JorgeHdzRiv/Desafios_Alura_DS/raw/refs/heads/main/ChatBot/models/modelo.zip"

# Descargar el archivo ZIP
os.system(f"wget {zip_url} -O modelo.zip")

# Descomprimir la carpeta
os.system("unzip -o modelo.zip")

# Eliminar el archivo ZIP después de extraer
os.system("rm modelo.zip")

# Verificar archivos descargados
os.system("ls ./modelo")

0

In [36]:
# Cargar el modelo entrenado y el tokenizer
ruta_modelo = './modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

# Diccionario de clases (asegúrate de que esté correctamente alineado con tu modelo)
diccionario = {0: 'Agradecimiento', 1: 'Aprendizaje', 2: 'Contacto', 3: 'Continuacion',
               4: 'Despedida', 5: 'Edad', 6: 'Error', 7: 'Funcion', 8: 'Identidad',
               9: 'Nombre', 10: 'Origen', 11: 'Otros', 12: 'Saludos', 13: 'Sentimiento',
               14: 'Usuario'}

# Función para procesar y clasificar la pregunta
def clasificacion_modelo(pregunta):
    frase = ' '.join(normalizar(pregunta))

    # Tokenizar la frase de entrada
    tokens = tokenizer_TF.encode_plus(
        frase,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # Obtener los input_ids y attention_mask
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    # Realizar la predicción
    with torch.no_grad():
        outputs = Modelo_TF(input_ids, attention_mask)

    # Obtener las etiquetas predichas
    etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

    # Decodificar las etiquetas predichas
    etiquetas_decodificadas = etiquetas_predichas.tolist()

    # Obtener la clase encontrada
    llave_buscada = etiquetas_decodificadas[0]
    clase_encontrada = diccionario[llave_buscada]
    print("Respuesta encontrada por el modelo Transformers", "se clasifica como: ", clase_encontrada)

    # Buscar la respuesta más parecida si no es 'Otros'
    if clase_encontrada != 'Otros':
        # Buscar respuesta más parecida en la clase encontrada
        df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
        df.reset_index(inplace=True)

        # Vectorización del texto
        vectorizer = TfidfVectorizer()
        dialogos_num = vectorizer.fit_transform(df['dialogo'])
        pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])

        # Calcular la similaridad
        similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
        indice_pregunta_proxima = similarity_scores.argmax()

        # Si la similaridad es mayor a 0.5, devolver la respuesta
        if max(similarity_scores) > 0.5:
            respuesta = df['respuesta'][indice_pregunta_proxima]
        else:
            respuesta = ''
            print('No se encontró una respuesta suficiente')

    else:
        respuesta = ''
        print('La clase es "Otros", por lo que no se devuelve respuesta.')

    return respuesta


## 5.3 Respuesta final del Chatbot

En la función respuesta_chatbot(pregunta) nuestro algoritmo buscará primero la respuesta por comparación de textos y sino la encuentra la buscará por el modelo entrenado de Machine Learning, ya está todo listo, veamos en el próximo paso como ejecutar nuestro Chatbot.

In [37]:
#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      return 'Respuesta no encontrada'

# 6.Ejecutar Chatbot

En este último paso sólo tenemos que relajarnos y dialogar con nuestro Chatbot, hazle preguntas que estén en los documentos de diálogo, varia un poco tus preguntas, pregunta cosas que el Chatbot no haya visto durante su entrenamiento, pruébalo y observa como nos responde, estamos creando una Inteligencia Artificial, pongamos a prueba esa inteligencia.

In [41]:
pregunta = 'hola como estas mi hermano'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el modelo Transformers se clasifica como:  Saludos
Estoy bien, gracias. ¿Y tú?


In [42]:
pregunta = 'Que es la inteligencia artificial'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
La inteligencia artificial (IA) es un campo de la informática que se centra en crear sistemas informáticos inteligentes. Estos sistemas se diseñan para simular y replicar la capacidad de solución de problemas de los humanos. Esto se logra mediante el uso de algoritmos y técnicas especializadas para permitir que los sistemas informáticos sean capaces de reconocer patrones, comunicarse, aprender, razonar y tomar decisiones.
